In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

tar: spark-2.4.1-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
     |████████████████████████████████| 204.2MB 56kB/s 
     |████████████████████████████████| 204kB 46.1MB/s 


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col,lit,regexp_replace,expr
from pyspark.sql.types import  StructField, StructType, StringType, IntegerType,FloatType
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler,MinMaxScaler
import numpy as np
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
spark = SparkSession.builder.appName("Modelo de Regreção").getOrCreate()
dataset = spark.read.csv("sample_data/houses_to_rent.csv",header=True)


In [ ]:
dataset.show()

+---+----+----+-----+--------+--------------+-----+---------+-------------+-------+-----------+------------+--------------+--------+
|_c0|city|area|rooms|bathroom|parking spaces|floor|   animal|    furniture|    hoa|rent amount|property tax|fire insurance|   total|
+---+----+----+-----+--------+--------------+-----+---------+-------------+-------+-----------+------------+--------------+--------+
|  0|   1| 240|    3|       3|             4|    -|    acept|    furnished|    R$0|    R$8,000|     R$1,000|         R$121| R$9,121|
|  1|   0|  64|    2|       1|             1|   10|    acept|not furnished|  R$540|      R$820|       R$122|          R$11| R$1,493|
|  2|   1| 443|    5|       5|             4|    3|    acept|    furnished|R$4,172|    R$7,000|     R$1,417|          R$89|R$12,680|
|  3|   1|  73|    2|       2|             1|   12|    acept|not furnished|  R$700|    R$1,250|       R$150|          R$16| R$2,116|
|  4|   1|  19|    1|       1|             0|    -|not acept|not furn

In [ ]:
dataset.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- city: string (nullable = true)
 |-- area: string (nullable = true)
 |-- rooms: string (nullable = true)
 |-- bathroom: string (nullable = true)
 |-- parking spaces: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- animal: string (nullable = true)
 |-- furniture: string (nullable = true)
 |-- hoa: string (nullable = true)
 |-- rent amount: string (nullable = true)
 |-- property tax: string (nullable = true)
 |-- fire insurance: string (nullable = true)
 |-- total: string (nullable = true)



In [ ]:
dataset = dataset.withColumnRenamed("_c0","ID")

In [ ]:
dataset.show()

+---+----+----+-----+--------+--------------+-----+---------+-------------+-------+-----------+------------+--------------+--------+
| ID|city|area|rooms|bathroom|parking spaces|floor|   animal|    furniture|    hoa|rent amount|property tax|fire insurance|   total|
+---+----+----+-----+--------+--------------+-----+---------+-------------+-------+-----------+------------+--------------+--------+
|  0|   1| 240|    3|       3|             4|    -|    acept|    furnished|    R$0|    R$8,000|     R$1,000|         R$121| R$9,121|
|  1|   0|  64|    2|       1|             1|   10|    acept|not furnished|  R$540|      R$820|       R$122|          R$11| R$1,493|
|  2|   1| 443|    5|       5|             4|    3|    acept|    furnished|R$4,172|    R$7,000|     R$1,417|          R$89|R$12,680|
|  3|   1|  73|    2|       2|             1|   12|    acept|not furnished|  R$700|    R$1,250|       R$150|          R$16| R$2,116|
|  4|   1|  19|    1|       1|             0|    -|not acept|not furn

In [ ]:
dataset.withColumn("hoa",regexp_replace(col("hoa"),"R\$","")).show()

+---+----+----+-----+--------+--------------+-----+---------+-------------+-----+-----------+------------+--------------+--------+
| ID|city|area|rooms|bathroom|parking spaces|floor|   animal|    furniture|  hoa|rent amount|property tax|fire insurance|   total|
+---+----+----+-----+--------+--------------+-----+---------+-------------+-----+-----------+------------+--------------+--------+
|  0|   1| 240|    3|       3|             4|    -|    acept|    furnished|    0|    R$8,000|     R$1,000|         R$121| R$9,121|
|  1|   0|  64|    2|       1|             1|   10|    acept|not furnished|  540|      R$820|       R$122|          R$11| R$1,493|
|  2|   1| 443|    5|       5|             4|    3|    acept|    furnished|4,172|    R$7,000|     R$1,417|          R$89|R$12,680|
|  3|   1|  73|    2|       2|             1|   12|    acept|not furnished|  700|    R$1,250|       R$150|          R$16| R$2,116|
|  4|   1|  19|    1|       1|             0|    -|not acept|not furnished|    0|  

In [ ]:
dataset = dataset.select([regexp_replace(col(c),"R\$","").alias(c) for c in dataset.columns])
dataset = dataset.select([regexp_replace(col(c),",","").alias(c) for c in dataset.columns])
dataset = dataset.select([regexp_replace(col(c),"not acept","0").alias(c) for c in dataset.columns])
dataset = dataset.select([regexp_replace(col(c),"acept","1").alias(c) for c in dataset.columns])
dataset = dataset.withColumn("furniture",regexp_replace(col("furniture"),"not furnished","0"))
dataset = dataset.withColumn("furniture",regexp_replace(col("furniture"),"furnished","1"))

In [ ]:
dataset.show()

+---+----+----+-----+--------+--------------+-----+------+---------+----+-----------+------------+--------------+-----+
| ID|city|area|rooms|bathroom|parking spaces|floor|animal|furniture| hoa|rent amount|property tax|fire insurance|total|
+---+----+----+-----+--------+--------------+-----+------+---------+----+-----------+------------+--------------+-----+
|  0|   1| 240|    3|       3|             4|    -|     1|        1|   0|       8000|        1000|           121| 9121|
|  1|   0|  64|    2|       1|             1|   10|     1|        0| 540|        820|         122|            11| 1493|
|  2|   1| 443|    5|       5|             4|    3|     1|        1|4172|       7000|        1417|            89|12680|
|  3|   1|  73|    2|       2|             1|   12|     1|        0| 700|       1250|         150|            16| 2116|
|  4|   1|  19|    1|       1|             0|    -|     0|        0|   0|       1200|          41|            16| 1257|
|  5|   1|  13|    1|       1|          

In [ ]:
lista_float = []
dataset = dataset.select([col(c).cast("Float") if c in lista_float else col(c).cast("Integer") for c in dataset.columns ])
dataset.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- city: integer (nullable = true)
 |-- area: integer (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- bathroom: integer (nullable = true)
 |-- parking spaces: integer (nullable = true)
 |-- floor: integer (nullable = true)
 |-- animal: integer (nullable = true)
 |-- furniture: integer (nullable = true)
 |-- hoa: integer (nullable = true)
 |-- rent amount: integer (nullable = true)
 |-- property tax: integer (nullable = true)
 |-- fire insurance: integer (nullable = true)
 |-- total: integer (nullable = true)



In [ ]:
dataset.show()

+---+----+----+-----+--------+--------------+-----+------+---------+----+-----------+------------+--------------+-----+
| ID|city|area|rooms|bathroom|parking spaces|floor|animal|furniture| hoa|rent amount|property tax|fire insurance|total|
+---+----+----+-----+--------+--------------+-----+------+---------+----+-----------+------------+--------------+-----+
|  0|   1| 240|    3|       3|             4| null|     1|        1|   0|       8000|        1000|           121| 9121|
|  1|   0|  64|    2|       1|             1|   10|     1|        0| 540|        820|         122|            11| 1493|
|  2|   1| 443|    5|       5|             4|    3|     1|        1|4172|       7000|        1417|            89|12680|
|  3|   1|  73|    2|       2|             1|   12|     1|        0| 700|       1250|         150|            16| 2116|
|  4|   1|  19|    1|       1|             0| null|     0|        0|   0|       1200|          41|            16| 1257|
|  5|   1|  13|    1|       1|          

In [ ]:
dataset.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in dataset.columns]).show()

+---+----+----+-----+--------+--------------+-----+------+---------+---+-----------+------------+--------------+-----+
| ID|city|area|rooms|bathroom|parking spaces|floor|animal|furniture|hoa|rent amount|property tax|fire insurance|total|
+---+----+----+-----+--------+--------------+-----+------+---------+---+-----------+------------+--------------+-----+
|  0|   0|   0|    0|       0|             0| 1555|     0|        0|202|          0|          27|             0|    0|
+---+----+----+-----+--------+--------------+-----+------+---------+---+-----------+------------+--------------+-----+



In [ ]:
dataset.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+
|summary|                ID|              city|              area|             rooms|          bathroom|    parking spaces|            floor|             animal|         furniture|               hoa|      rent amount|     property tax|   fire insurance|            total|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+
|  count|              6080|              6080|              6080|              6080|              6080|              6080|             4525|               6080|              6080|    

In [ ]:
dataset = dataset.drop("floor")

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset.show()

+---+----+----+-----+--------+--------------+------+---------+----+-----------+------------+--------------+-----+
| ID|city|area|rooms|bathroom|parking spaces|animal|furniture| hoa|rent amount|property tax|fire insurance|total|
+---+----+----+-----+--------+--------------+------+---------+----+-----------+------------+--------------+-----+
|  0|   1| 240|    3|       3|             4|     1|        1|   0|       8000|        1000|           121| 9121|
|  1|   0|  64|    2|       1|             1|     1|        0| 540|        820|         122|            11| 1493|
|  2|   1| 443|    5|       5|             4|     1|        1|4172|       7000|        1417|            89|12680|
|  3|   1|  73|    2|       2|             1|     1|        0| 700|       1250|         150|            16| 2116|
|  4|   1|  19|    1|       1|             0|     0|        0|   0|       1200|          41|            16| 1257|
|  5|   1|  13|    1|       1|             0|     1|        0|   0|       2200|         

In [ ]:
x = dataset.drop("total")

In [ ]:
ass = VectorAssembler(inputCols=x.columns, outputCol="features")
dataset = ass.transform(dataset)


In [ ]:
dataset = dataset.withColumnRenamed("total","label")

In [ ]:
dataset.show()

+---+----+----+-----+--------+--------------+------+---------+----+-----------+------------+--------------+-----+--------------------+
| ID|city|area|rooms|bathroom|parking spaces|animal|furniture| hoa|rent amount|property tax|fire insurance|label|            features|
+---+----+----+-----+--------+--------------+------+---------+----+-----------+------------+--------------+-----+--------------------+
|  0|   1| 240|    3|       3|             4|     1|        1|   0|       8000|        1000|           121| 9121|[0.0,1.0,240.0,3....|
|  1|   0|  64|    2|       1|             1|     1|        0| 540|        820|         122|            11| 1493|[1.0,0.0,64.0,2.0...|
|  2|   1| 443|    5|       5|             4|     1|        1|4172|       7000|        1417|            89|12680|[2.0,1.0,443.0,5....|
|  3|   1|  73|    2|       2|             1|     1|        0| 700|       1250|         150|            16| 2116|[3.0,1.0,73.0,2.0...|
|  4|   1|  19|    1|       1|             0|     0|   

In [ ]:
featureScaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures").fit(dataset)

In [ ]:
(treino, teste) = dataset.randomSplit([0.7, 0.3])

In [ ]:
dt = GBTRegressor(featuresCol="scaledFeatures")

pipeline = Pipeline(stages=[featureScaler, dt])

model = pipeline.fit(treino)

pred = model.transform(teste)

In [ ]:
pred.select("prediction", "label", "features").show(15)

avaliador = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")

mae = avaliador.evaluate(pred)

print("Erro medio absoluto nos dados de teste = %g" % mae)

GBT = model.stages[1]
print(GBT)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|  9456.07182231054| 9121|[0.0,1.0,240.0,3....|
|1432.2770292031184| 1493|[1.0,0.0,64.0,2.0...|
| 2649.208007140318| 2116|[3.0,1.0,73.0,2.0...|
|1613.2700339579374| 1984|[7.0,1.0,55.0,2.0...|
|4854.8366856911325| 4691|[14.0,1.0,92.0,2....|
|1362.8475957971316| 1319|[15.0,0.0,56.0,2....|
|3572.2207905979053| 3813|[17.0,1.0,100.0,3...|
|2507.3115667738653| 2923|[20.0,1.0,155.0,3...|
|4056.3621644682917| 4452|[22.0,1.0,65.0,2....|
|3590.0537491097834| 3577|[25.0,1.0,31.0,1....|
|15100.852021526594|17260|[27.0,1.0,300.0,3...|
| 6230.835644332444| 6175|[29.0,1.0,130.0,3...|
|  19010.3427742286|19100|[39.0,1.0,191.0,3...|
|24948.272073552045|24530|[41.0,1.0,800.0,4...|
|3456.4576450967006| 3000|[45.0,1.0,100.0,2...|
+------------------+-----+--------------------+
only showing top 15 rows

Erro medio absoluto nos dados de teste = 1025.36
GBTRegression